In [5]:
from transformers import AutoTokenizer
import json
from tqdm import tqdm

/home/nlp_team/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
import pandas as pd
from datasets import Dataset, DatasetDict

with open("vosint.json", encoding='utf-8') as f:
    vosint = json.load(f)

train_json = vosint['train']
test_json = vosint['test']
# with open("test.json", encoding='utf-8') as f:
#     test_dataset = json.load(f)

dataset = {}
dataset
mapping = {
    "tieu_cuc": 0,
    "trung_tinh": 1,
    "tich_cuc": 2
}
train_df = pd.DataFrame.from_dict(train_json)
test_df  = pd.DataFrame.from_dict(test_json)

dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

dataset = DatasetDict()
dataset['train'] = dataset_train
dataset['test'] = dataset_test

dataset.sa
# train = [] 
# for record in train_dataset:
#     new_record = {}
#     new_record['text'] = record['title'].strip()+". "+record.get('description',"").strip()+record['text'].strip()
#     new_record['label'] = mapping[record['label'][-1]]
#     train.append(new_record)

# test = [] 
# for record in test_dataset:
#     new_record = {}
#     new_record['text'] = record['title'].strip()+". "+record.get('description',"").strip()+record['text'].strip()
#     new_record['label'] = mapping[record['label'][-1]]
#     test.append(new_record)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2224
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1002
    })
})

In [7]:
rdrsegmenter

In [14]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
dataset = train_dataset+test_dataset
len(dataset)
count=0
len_token = []
for record in tqdm(dataset):
    text =  record['title'].strip()+record.get('description',"").strip()+record['text'].strip()
    text = " ".join(rdrsegmenter.word_segment(text))
    token = tokenizer(text)
    len_token.append(len(token['input_ids']))
    if len(token['input_ids']) > 512:
        count+=1
print(count/len(dataset))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 3226/3226 [00:28<00:00, 111.77it/s]

0.5390576565406076


In [13]:
print(count/len(dataset))

0.49566026038437694


In [11]:
import torch
a = torch.Tensor(len_token)
a.mean()

tensor(647.4647)

In [1]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='./')

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/media/nlp_team/nlpteam/toan/bert_vosint/dataset')
# text = """Trong thời gian vừa qua, câu chuyện hiệu trưởng trường Đại học Tôn Đức Thắng bị kỷ luật cách chức đã và đang được sự quan tâm rất lớn của dư luận xã hội, cử tri cả nước, cũng như đã làm nóng phiên chất vấn của Quốc hội vừa qua.\nPhóng viên Dân trí đã có cuộc trao đổi với TS. Nguyễn Sĩ Dũng, Nguyên Phó chủ nhiệm Văn phòng Quốc hội về vấn đề này.\nPhóng viên: Thành tích của Đại học Tôn Đức Thắng (TDTU) đã được nhiều nhà lãnh đạo, nhà quản lý, đối tác và các tổ chức xếp hạng uy tín quốc tế đánh giá cao, được xem là điểm sáng của tự chủ đại học Việt Nam nhưng hiệu trưởng của trường này vừa bị kỷ luật cách chức. Ông có suy nghĩ gì về việc này?\nTS. Nguyễn Sĩ Dũng: Quả thật, tôi thấy rất băn khoăn. Tự chủ đại học là chủ trương lớn của Đảng, đã được cụ thể hóa trong Luật sửa đổi, bổ sung một số điều của Luật giáo dục đại học và trong Nghị định 99/2019/NĐ-CP.\nTrường Đại học Tôn Đức Thắng đã đi tiên phong thực hiện thí điểm tự chủ đại học và đã đạt kết quả tích cực. Trong cùng một thời điểm, các tổ chức xếp hạng quốc tế đánh giá TDTU có thứ hạng trong TOP 800 (AWRU) và 700 (US News) trường đại học tốt nhất thế giới và là cơ sở giáo dục đại học số 1 của Việt Nam.\nĐây quả thật là một thành tích mà ngành giáo dục đại học của nước ta, cũng như TP HCM có thể tự hào. Chính vì vậy, mọi sự việc liên quan đến TDTU cần được xem xét một cách hết sức cẩn trọng. Kỷ luật Hiệu trưởng của trường trong bối cảnh như nói trên dễ làm cho dư luận dị nghị vì điều này rất khó hiểu và rất bất hợp lý.\nNgoài ra, cũng cần thấy rằng thí điểm là làm theo cách mới, theo chuẩn mực mới. Áp đặt cách cũ, chuẩn mực cũ thì còn thí điểm cái gì?\nQua theo dõi vụ việc Hiệu trưởng TDTU bị kỷ luật trên phương tiện truyền thông, tôi thấy sai phạm được nêu ra không có gì thật cụ thể và kết luận vi phạm cũng rất chung chung.\nTôi được biết ông Lê Vinh Danh, cựu Hiệu trưởng của TDTU cũng đang có khiếu nại về quyết định kỷ luật Đảng và quyết định kỷ luật cách chức đối với ông.\nPhóng viên: Tại phiên chất vấn sáng ngày 6/11 của Quốc hội, Đại biểu Lê Thanh Vân đã nêu câu hỏi và 2 lần tranh luận xung quanh việc cách chức hiệu trưởng TDTU, ông có đánh giá gì về vấn đề này?\nTS. Nguyễn Sĩ Dũng: Trước hết, việc đại biểu Quốc hội chất vấn ở diễn đàn của Quốc hội cho thấy đây là vấn đề có tầm quan trọng đối với quốc gia và là vấn đề đang được dư luận xã hội quan tâm.\nThứ hai, tôi ủng hộ quan điểm của đại biểu Quốc hội Lê Thanh Vân. Việc cách chức ông Lê Vinh Danh phải tuân thủ các quy định của pháp luật. \nLuật Giáo dục đại học và Nghị định 99/2019 hướng dẫn thi hành đều quy định rất rõ về thẩm quyền của hội đồng trường, của cơ quan chủ quản trong việc quyết định nhân sự chức danh hiệu trưởng, cũng như thủ tục để tiến hành công việc này. Quan trọng là phải tuân thủ các quy định nói trên, thì việc cách chức ông Lê Vinh Danh mới hợp pháp được.\nThậm chí khuyết hội đồng trường cũng không phải là lý do để cơ quan chủ quản có thể trực tiếp cách chức hiệu trưởng.\nĐiểm a Khoản 6 Điều 7 của Nghị định 99/2019 quy định: “Trường hợp nhiệm kỳ của hiệu trưởng kết thúc đồng thời với nhiệm kỳ của hội đồng trường thì hiệu trưởng được kéo dài thời gian thực hiện nhiệm vụ cho đến khi cơ quan quản lý trực tiếp công nhận hiệu trưởng của nhiệm kỳ kế tiếp, trên cơ sở đề nghị của hội đồng trường nhiệm kỳ kế tiếp”. Như vậy, việc cách chức ông Lê Vinh Danh là rõ ràng là chưa tuân thủ quy định này của pháp luật.\nTôi cũng tán thành lập luận của đại biểu Lê Thanh Vân là: khi xử lý một vấn đề pháp lý, chúng ta cần phải áp dụng văn bản luật chuyên ngành trước, chỉ khi nào không có luật chuyên ngành hoặc luật chuyên ngành không quy định thì mới áp dụng văn bản pháp luật khác có liên quan.\nVới thực tế như trên trên, tôi cho rằng cơ quan chủ quản nên xem xét lại quyết định cách chức ông Lê Vinh Danh của mình để tránh bị kiện tụng và tránh gây ra một cuộc khủng hoảng truyền thông không đáng có.\nPhóng viên: Từ sự việc này, theo ông cần rút ra bài học gì cho các cơ quan, tổ chức có trường đại học  đang vận dụng mô hình tự chủ?\nTS. Nguyễn Sĩ Dũng: Tự chủ đại học là chủ trương lớn, phản ánh tầm nhìn chiến lược của Đảng và Nhà nước ta. Tuy nhiên, đây là một công việc rất mới, đồng thời rất khó. Để hiện thực hóa chủ trương này, những người dám dấn thân, dám đi tiên phong như ông Lê Vinh Danh cần phải được bảo vệ.\nPhóng viên: Xin trân trọng cảm ơn ông!"""
# output = rdrsegmenter(text)
# output

2023-07-06 09:45:10 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [4]:
text = """Trong thời gian vừa qua, câu chuyện hiệu trưởng trường Đại học Tôn Đức Thắng bị kỷ luật cách chức đã và đang được sự quan tâm rất lớn của dư luận xã hội, cử tri cả nước, cũng như đã làm nóng phiên chất vấn của Quốc hội vừa qua.\nPhóng viên Dân trí đã có cuộc trao đổi với TS. Nguyễn Sĩ Dũng, Nguyên Phó chủ nhiệm Văn phòng Quốc hội về vấn đề này.\nPhóng viên: Thành tích của Đại học Tôn Đức Thắng (TDTU) đã được nhiều nhà lãnh đạo, nhà quản lý, đối tác và các tổ chức xếp hạng uy tín quốc tế đánh giá cao, được xem là điểm sáng của tự chủ đại học Việt Nam nhưng hiệu trưởng của trường này vừa bị kỷ luật cách chức. Ông có suy nghĩ gì về việc này?\nTS. Nguyễn Sĩ Dũng: Quả thật, tôi thấy rất băn khoăn. Tự chủ đại học là chủ trương lớn của Đảng, đã được cụ thể hóa trong Luật sửa đổi, bổ sung một số điều của Luật giáo dục đại học và trong Nghị định 99/2019/NĐ-CP.\nTrường Đại học Tôn Đức Thắng đã đi tiên phong thực hiện thí điểm tự chủ đại học và đã đạt kết quả tích cực. Trong cùng một thời điểm, các tổ chức xếp hạng quốc tế đánh giá TDTU có thứ hạng trong TOP 800 (AWRU) và 700 (US News) trường đại học tốt nhất thế giới và là cơ sở giáo dục đại học số 1 của Việt Nam.\nĐây quả thật là một thành tích mà ngành giáo dục đại học của nước ta, cũng như TP HCM có thể tự hào. Chính vì vậy, mọi sự việc liên quan đến TDTU cần được xem xét một cách hết sức cẩn trọng. Kỷ luật Hiệu trưởng của trường trong bối cảnh như nói trên dễ làm cho dư luận dị nghị vì điều này rất khó hiểu và rất bất hợp lý.\nNgoài ra, cũng cần thấy rằng thí điểm là làm theo cách mới, theo chuẩn mực mới. Áp đặt cách cũ, chuẩn mực cũ thì còn thí điểm cái gì?\nQua theo dõi vụ việc Hiệu trưởng TDTU bị kỷ luật trên phương tiện truyền thông, tôi thấy sai phạm được nêu ra không có gì thật cụ thể và kết luận vi phạm cũng rất chung chung.\nTôi được biết ông Lê Vinh Danh, cựu Hiệu trưởng của TDTU cũng đang có khiếu nại về quyết định kỷ luật Đảng và quyết định kỷ luật cách chức đối với ông.\nPhóng viên: Tại phiên chất vấn sáng ngày 6/11 của Quốc hội, Đại biểu Lê Thanh Vân đã nêu câu hỏi và 2 lần tranh luận xung quanh việc cách chức hiệu trưởng TDTU, ông có đánh giá gì về vấn đề này?\nTS. Nguyễn Sĩ Dũng: Trước hết, việc đại biểu Quốc hội chất vấn ở diễn đàn của Quốc hội cho thấy đây là vấn đề có tầm quan trọng đối với quốc gia và là vấn đề đang được dư luận xã hội quan tâm.\nThứ hai, tôi ủng hộ quan điểm của đại biểu Quốc hội Lê Thanh Vân. Việc cách chức ông Lê Vinh Danh phải tuân thủ các quy định của pháp luật. \nLuật Giáo dục đại học và Nghị định 99/2019 hướng dẫn thi hành đều quy định rất rõ về thẩm quyền của hội đồng trường, của cơ quan chủ quản trong việc quyết định nhân sự chức danh hiệu trưởng, cũng như thủ tục để tiến hành công việc này. Quan trọng là phải tuân thủ các quy định nói trên, thì việc cách chức ông Lê Vinh Danh mới hợp pháp được.\nThậm chí khuyết hội đồng trường cũng không phải là lý do để cơ quan chủ quản có thể trực tiếp cách chức hiệu trưởng.\nĐiểm a Khoản 6 Điều 7 của Nghị định 99/2019 quy định: “Trường hợp nhiệm kỳ của hiệu trưởng kết thúc đồng thời với nhiệm kỳ của hội đồng trường thì hiệu trưởng được kéo dài thời gian thực hiện nhiệm vụ cho đến khi cơ quan quản lý trực tiếp công nhận hiệu trưởng của nhiệm kỳ kế tiếp, trên cơ sở đề nghị của hội đồng trường nhiệm kỳ kế tiếp”. Như vậy, việc cách chức ông Lê Vinh Danh là rõ ràng là chưa tuân thủ quy định này của pháp luật.\nTôi cũng tán thành lập luận của đại biểu Lê Thanh Vân là: khi xử lý một vấn đề pháp lý, chúng ta cần phải áp dụng văn bản luật chuyên ngành trước, chỉ khi nào không có luật chuyên ngành hoặc luật chuyên ngành không quy định thì mới áp dụng văn bản pháp luật khác có liên quan.\nVới thực tế như trên trên, tôi cho rằng cơ quan chủ quản nên xem xét lại quyết định cách chức ông Lê Vinh Danh của mình để tránh bị kiện tụng và tránh gây ra một cuộc khủng hoảng truyền thông không đáng có.\nPhóng viên: Từ sự việc này, theo ông cần rút ra bài học gì cho các cơ quan, tổ chức có trường đại học  đang vận dụng mô hình tự chủ?\nTS. Nguyễn Sĩ Dũng: Tự chủ đại học là chủ trương lớn, phản ánh tầm nhìn chiến lược của Đảng và Nhà nước ta. Tuy nhiên, đây là một công việc rất mới, đồng thời rất khó. Để hiện thực hóa chủ trương này, những người dám dấn thân, dám đi tiên phong như ông Lê Vinh Danh cần phải được bảo vệ.\nPhóng viên: Xin trân trọng cảm ơn ông!"""
output = rdrsegmenter.word_segment(text)


['Trong thời_gian vừa_qua , câu_chuyện hiệu_trưởng trường Đại_học Tôn_Đức_Thắng bị kỷ_luật cách_chức đã và đang được sự quan_tâm rất lớn của dư_luận xã_hội , cử_tri cả nước , cũng như đã làm nóng phiên chất_vấn của Quốc_hội vừa_qua .',
 'Phóng_viên Dân_trí đã có cuộc trao_đổi với TS .',
 'Nguyễn_Sĩ_Dũng , Nguyên Phó chủ_nhiệm Văn_phòng Quốc_hội về vấn_đề này .',
 'Phóng_viên : Thành_tích của Đại_học Tôn_Đức_Thắng ( TDTU ) đã được nhiều nhà_lãnh_đạo , nhà_quản_lý , đối_tác và các tổ_chức xếp_hạng uy_tín quốc_tế đánh_giá cao , được xem là điểm sáng của tự_chủ đại_học Việt_Nam nhưng hiệu_trưởng của trường này vừa bị kỷ_luật cách_chức .',
 'Ông có suy_nghĩ gì về việc này ?',
 'TS .',
 'Nguyễn_Sĩ_Dũng : Quả_thật , tôi thấy rất băn_khoăn .',
 'Tự_chủ đại_học là chủ_trương lớn của Đảng , đã được cụ_thể_hoá trong Luật sửa_đổi , bổ_sung một_số điều của Luật_giáo_dục đại_học và trong Nghị_định 99/2019/NĐ-CP. Trường Đại_học Tôn_Đức_Thắng đã đi tiên_phong thực_hiện thí_điểm tự_chủ đại_học và đã đạ